In [ ]:
# =========================================================
# KBO Next Season Prediction (Deep Learning MLP) - DELTA MODEL (FINAL)
# ✅ 2015년 이후만 사용
# ✅ t시즌 PA >= 223 (입력 안정화)
# ✅ 타깃은 "다음시즌 절대값"이 아니라 "증감(Δ = next - current)" 예측
# ✅ 예측 시: next_pred = current + delta_pred
# ✅ 저장: model_kbo/ (app에서 로드 가능)
# =========================================================

import os
import joblib
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

# -------------------------
# 0) Seed / Path
# -------------------------
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

CSV_PATH = r"C:\Users\yusan\OneDrive\Desktop\2025 winter\Notebook\Colab Notebooks\dataset\kbo_batting_stats.csv"

# -------------------------
# 1) Config
# -------------------------
ID_COL   = "Id"
YEAR_COL = "Year"
AGE_COL  = "Age"
PA_COL   = "PA"

TRAIN_MIN_YEAR = 2015
PA_MIN_T       = 223

#타겟 
TARGETS = ["AVG", "RBI", "HR", "SB", "OBP", "SLG", "OPS", "WAR", "wRC_plus"]

# 데이터 로드
df0 = pd.read_csv(CSV_PATH)
print("Loaded:", df0.shape)

# 컬럼명 안전 처리
df0 = df0.rename(columns={"wRC+": "wRC_plus"})

# 필수 컬럼 체크
required = [ID_COL, YEAR_COL, AGE_COL, PA_COL] + TARGETS
missing_req = [c for c in required if c not in df0.columns]
if missing_req:
    raise ValueError(f"필수 컬럼이 없습니다: {missing_req}\n현재 컬럼: {list(df0.columns)}")

# Numeric 변환 + 정렬
df0[YEAR_COL] = pd.to_numeric(df0[YEAR_COL], errors="coerce")
df0[AGE_COL]  = pd.to_numeric(df0[AGE_COL], errors="coerce")
df0[PA_COL]   = pd.to_numeric(df0[PA_COL], errors="coerce")

for t in TARGETS:
    df0[t] = pd.to_numeric(df0[t], errors="coerce")

df0 = df0.dropna(subset=[ID_COL, YEAR_COL, AGE_COL, PA_COL]).copy()
df0 = df0.sort_values([ID_COL, YEAR_COL]).reset_index(drop=True)

# 시즌 기준 필터링
before = len(df0)
df0 = df0[(df0[YEAR_COL] >= TRAIN_MIN_YEAR) & (df0[PA_COL] >= PA_MIN_T)].copy()
after = len(df0)
print(f"[Filter] YEAR>={TRAIN_MIN_YEAR} & PA>={PA_MIN_T}: {before} -> {after} (removed {before-after})")

# 예측값 만들기 
df0[f"{PA_COL}_next"] = df0.groupby(ID_COL)[PA_COL].shift(-1)
df0[f"{YEAR_COL}_next"] = df0.groupby(ID_COL)[YEAR_COL].shift(-1)

for t in TARGETS:
    df0[f"{t}_next"] = df0.groupby(ID_COL)[t].shift(-1)

# prev을 통한 보조 feature 생성
df0[f"{PA_COL}_prev"] = df0.groupby(ID_COL)[PA_COL].shift(1)
for t in TARGETS:
    df0[f"{t}_prev"] = df0.groupby(ID_COL)[t].shift(1)

# 학습 샘플 구성 
need_next = [f"{t}_next" for t in TARGETS] + [f"{PA_COL}_next"]
df = df0.dropna(subset=need_next + [YEAR_COL, AGE_COL, PA_COL]).copy()

# 숫자형 강제 + 결측 정리
for c in TARGETS + [f"{t}_next" for t in TARGETS] + [PA_COL, f"{PA_COL}_next", f"{PA_COL}_prev"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

df = df.dropna(subset=TARGETS + [f"{t}_next" for t in TARGETS] + [PA_COL, f"{PA_COL}_next"]).copy()

print("Years in dataset:", int(df[YEAR_COL].min()), "~", int(df[YEAR_COL].max()))

# Δ 타깃 만들기
for t in TARGETS:
    df[f"{t}_delta"] = df[f"{t}_next"] - df[t]

df[f"{PA_COL}_delta"] = df[f"{PA_COL}_next"] - df[PA_COL]  # 참고용(출장 증감)

DELTA_TARGETS = [f"{t}_delta" for t in TARGETS] + [f"{PA_COL}_delta"]

# Feature 구성
#    - 나이(에이징), 이번 시즌 스탯, PA, (prev 스탯/PA는 보조)
df["Age2"] = df[AGE_COL] ** 2

feature_cols = (
    [AGE_COL, "Age2", PA_COL]
    + TARGETS
    + [f"{PA_COL}_prev"] + [f"{t}_prev" for t in TARGETS]
)

X = df[feature_cols].values.astype("float32")
Y = df[DELTA_TARGETS].values.astype("float32")

print("Final X/Y:", X.shape, Y.shape)
print("Delta targets:", DELTA_TARGETS)

# Time-based split (마지막 2년 test, 부족하면 자동 확장)
years_sorted = np.array(sorted(df[YEAR_COL].dropna().unique()))
max_year = int(years_sorted.max())

def make_split(_df, start_test_year):
    train_mask = (_df[YEAR_COL] < start_test_year)
    test_mask  = (_df[YEAR_COL] >= start_test_year)
    return train_mask, test_mask

start_test_year = max_year - 1
for _ in range(6):
    train_mask, test_mask = make_split(df, start_test_year)
    if train_mask.sum() > 0 and test_mask.sum() > 0:
        break
    start_test_year -= 1

print(f"[Split] train < {start_test_year}, test >= {start_test_year}")
print("Train samples:", int(train_mask.sum()), "Test samples:", int(test_mask.sum()))
if train_mask.sum() == 0 or test_mask.sum() == 0:
    raise ValueError("train/test 분할 실패: 필터가 너무 강함. PA나 연도 기준을 완화하세요.")

X_train, Y_train = X[train_mask], Y[train_mask]
X_test,  Y_test  = X[test_mask],  Y[test_mask]

# Impute + Scale (X/Y 분리)
x_imputer = SimpleImputer(strategy="median")
x_scaler  = StandardScaler()

y_scaler  = StandardScaler() 

X_train_s = x_scaler.fit_transform(x_imputer.fit_transform(X_train)).astype("float32")
X_test_s  = x_scaler.transform(x_imputer.transform(X_test)).astype("float32")

Y_train_s = y_scaler.fit_transform(Y_train).astype("float32")
Y_test_s  = y_scaler.transform(Y_test).astype("float32")

# MLP 빌드
def build_mlp(n_features, n_targets):
    inputs = keras.Input(shape=(n_features,))
    x = layers.Dense(256, activation="relu")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.25)(x)

    x = layers.Dense(128, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.20)(x)

    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.10)(x)

    outputs = layers.Dense(n_targets, activation="linear")(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss="mse",
        metrics=[keras.metrics.MeanAbsoluteError(name="mae")]
    )
    return model

model = build_mlp(X_train_s.shape[1], Y_train_s.shape[1])
model.summary()

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
]

# 샘플 가중치: PA 큰 시즌을 더 신뢰 (노이즈 완화)
sample_weight = df.loc[train_mask, PA_COL].values.astype("float32")
sample_weight = np.clip(sample_weight, 1.0, None)

history = model.fit(
    X_train_s, Y_train_s,
    sample_weight=sample_weight,
    validation_split=0.2,
    epochs=120,
    batch_size=256,
    callbacks=callbacks,
    verbose=1
)

# -------------------------
# 12) 학습 평가
pred_delta_s = model.predict(X_test_s, verbose=0)
pred_delta   = y_scaler.inverse_transform(pred_delta_s)
true_delta   = y_scaler.inverse_transform(Y_test_s)

print("\n=== Test Metrics (DELTA) ===")
for i, col in enumerate(DELTA_TARGETS):
    mae  = mean_absolute_error(true_delta[:, i], pred_delta[:, i])
    rmse = float(np.sqrt(mean_squared_error(true_delta[:, i], pred_delta[:, i])))
    print(f"{col:12s} | MAE: {mae:.4f} | RMSE: {rmse:.4f}")

# 현재값(current) = df의 test rows에서 가져옴
df_test = df.loc[test_mask].copy()

# current matrix (TARGETS + PA)
current_cols = TARGETS + [PA_COL]
current = df_test[current_cols].values.astype("float32")

# true next matrix
true_next_cols = [f"{t}_next" for t in TARGETS] + [f"{PA_COL}_next"]
true_next = df_test[true_next_cols].values.astype("float32")

# pred next = current + pred_delta
pred_next = current + pred_delta  

print("\n=== Test Metrics (RECONSTRUCTED NEXT = current + delta_pred) ===")
next_names = [f"{t}_next" for t in TARGETS] + [f"{PA_COL}_next"]
for i, name in enumerate(next_names):
    mae  = mean_absolute_error(true_next[:, i], pred_next[:, i])
    rmse = float(np.sqrt(mean_squared_error(true_next[:, i], pred_next[:, i])))
    print(f"{name:12s} | MAE: {mae:.4f} | RMSE: {rmse:.4f}")

# -------------------------
# 13) Save bundle (앱에서 사용)
# -------------------------
os.makedirs("model_kbo", exist_ok=True)

model.save("model_kbo/kbo_mlp.keras")
joblib.dump(x_imputer, "model_kbo/imputer.pkl")
joblib.dump(x_scaler,  "model_kbo/x_scaler.pkl")
joblib.dump(y_scaler,  "model_kbo/y_scaler.pkl")

joblib.dump(feature_cols, "model_kbo/feature_cols.pkl")

joblib.dump(next_names, "model_kbo/targets.pkl")

joblib.dump(
    {
        "mode": "delta",
        "train_min_year": TRAIN_MIN_YEAR,
        "pa_min_t": PA_MIN_T,
        "split_start_test_year": int(start_test_year),
        "targets": TARGETS,
        "delta_targets": DELTA_TARGETS,
        "current_cols": current_cols,
        "next_names": next_names,
        "note": "Model predicts deltas. At inference: next_pred = current + delta_pred.",
    },
    "model_kbo/meta.pkl"
)

print("\nSaved to ./model_kbo/")

# Inference helper (노트북에서 바로 확인용)
def predict_next_for_player(player_id, base_year=2025):
    row = df0[(df0[ID_COL] == player_id) & (df0[YEAR_COL].astype(int) == int(base_year))].copy()
    if len(row) != 1:
        raise ValueError(f"player_id={player_id}, year={base_year} 행을 1개로 찾지 못함 (찾은 개수={len(row)})")

    # 파생: Age2
    row["Age2"] = row[AGE_COL].astype(float) ** 2

    prev = df0[(df0[ID_COL] == player_id) & (df0[YEAR_COL].astype(int) == int(base_year) - 1)].copy()
    if len(prev) == 1:
        row[f"{PA_COL}_prev"] = float(prev.iloc[0][PA_COL])
        for t in TARGETS:
            row[f"{t}_prev"] = float(prev.iloc[0][t])
    else:
        row[f"{PA_COL}_prev"] = np.nan
        for t in TARGETS:
            row[f"{t}_prev"] = np.nan

    x_raw = row[feature_cols].values.astype("float32")
    x_raw = np.nan_to_num(x_raw, nan=np.nan)

    x_s = x_scaler.transform(x_imputer.transform(x_raw)).astype("float32")
    delta_s = model.predict(x_s, verbose=0)
    delta = y_scaler.inverse_transform(delta_s)[0]  # (targets_delta + PA_delta)

    # current
    cur = np.array([float(row.iloc[0][c]) for c in current_cols], dtype="float32")

    next_pred = cur + delta

    # 결과 dict
    out = {}
    for i, t in enumerate(TARGETS):
        out[f"{t}_next_pred"] = float(next_pred[i])
        out[f"{t}_delta_pred"] = float(delta[i])

    out[f"{PA_COL}_next_pred"] = float(max(0.0, next_pred[-1]))
    out[f"{PA_COL}_delta_pred"] = float(delta[-1])

    return out

print("\n" + "="*80)
print("[A] FEATURE LIST USED FOR TRAINING (feature_cols)")
print("="*80)
print(f"Total features: {len(feature_cols)}")
for i, c in enumerate(feature_cols, 1):
    print(f"{i:3d}. {c}")

# feature 결측률도 확인 (데이터 품질 체크)
print("\n" + "="*80)
print("[B] FEATURE MISSING RATE (on training dataframe 'df')")
print("="*80)

missing_rate = (df[feature_cols].isna().mean().sort_values(ascending=False) * 100).round(2)
missing_df = missing_rate.reset_index()
missing_df.columns = ["feature", "missing_%"]
display(missing_df.head(30))  # 결측 많은 상위 30개
print("\n(참고) 결측 상위 30개만 표시했음. 전체는 missing_df 변수에 있음.")

# 표준화 전(원본) 분포 간단 요약도 같이
print("\n" + "="*80)
print("[C] FEATURE SUMMARY (raw df[feature_cols])")
print("="*80)
desc = df[feature_cols].describe().T
display(desc[["count","mean","std","min","25%","50%","75%","max"]].head(30))
print("\n(참고) 상위 30개만 표시. 전체는 desc 변수에 있음.")

# =========================================================
# [D] EVALUATION SUMMARY TABLES
#   - 1) DELTA 성능 (모델이 직접 예측한 값)
#   - 2) NEXT 성능 (current + delta_pred)
#   - 3) BASELINE 비교 (delta=0, 즉 next=current)
# =========================================================

def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

required_vars = ["pred_delta", "true_delta", "pred_next", "true_next", "DELTA_TARGETS", "next_names", "current"]
missing_vars = [v for v in required_vars if v not in globals()]
if missing_vars:
    raise NameError(f"아래 변수가 없어서 요약 리포트 생성 불가: {missing_vars}\n"
                    f"-> 12) Evaluate 셀이 먼저 실행되어야 함.")

# 1) DELTA metrics table
rows = []
for i, name in enumerate(DELTA_TARGETS):
    y_t = true_delta[:, i]
    y_p = pred_delta[:, i]
    rows.append({
        "target": name,
        "MAE": float(mean_absolute_error(y_t, y_p)),
        "RMSE": rmse(y_t, y_p),
    })
delta_metrics_df = pd.DataFrame(rows).sort_values("RMSE", ascending=True)
print("\n" + "="*80)
print("[D-1] TEST METRICS (DELTA)")
print("="*80)
display(delta_metrics_df)
print("DELTA avg | MAE:", float(delta_metrics_df["MAE"].mean()), "| RMSE:", float(delta_metrics_df["RMSE"].mean()))

# 2) NEXT metrics table (reconstructed)
rows = []
for i, name in enumerate(next_names):
    y_t = true_next[:, i]
    y_p = pred_next[:, i]
    rows.append({
        "target": name,
        "MAE": float(mean_absolute_error(y_t, y_p)),
        "RMSE": rmse(y_t, y_p),
    })
next_metrics_df = pd.DataFrame(rows).sort_values("RMSE", ascending=True)
print("\n" + "="*80)
print("[D-2] TEST METRICS (NEXT = current + delta_pred)")
print("="*80)
display(next_metrics_df)
print("NEXT avg | MAE:", float(next_metrics_df["MAE"].mean()), "| RMSE:", float(next_metrics_df["RMSE"].mean()))

# 3) BASELINE: delta=0 -> next_pred_baseline = current
# (즉, "내년은 올해랑 같다"라는 가장 단순한 기준)
baseline_pred_next = current.copy()

rows = []
for i, name in enumerate(next_names):
    y_t = true_next[:, i]
    y_base = baseline_pred_next[:, i]
    y_model = pred_next[:, i]

    base_mae = float(mean_absolute_error(y_t, y_base))
    base_rmse = rmse(y_t, y_base)
    model_mae = float(mean_absolute_error(y_t, y_model))
    model_rmse = rmse(y_t, y_model)

    rows.append({
        "target": name,
        "BASE_MAE": base_mae,
        "MODEL_MAE": model_mae,
        "MAE_improve_%": float((base_mae - model_mae) / base_mae * 100) if base_mae != 0 else np.nan,
        "BASE_RMSE": base_rmse,
        "MODEL_RMSE": model_rmse,
        "RMSE_improve_%": float((base_rmse - model_rmse) / base_rmse * 100) if base_rmse != 0 else np.nan,
    })

baseline_df = pd.DataFrame(rows)
baseline_df = baseline_df.sort_values("RMSE_improve_%", ascending=False)

print("\n" + "="*80)
print("[D-3] BASELINE COMPARISON (baseline: next=current, i.e., delta=0)")
print("="*80)
display(baseline_df)

print("\nSummary (mean improvement):")
print("MAE improvement %:", float(baseline_df["MAE_improve_%"].mean()))
print("RMSE improvement %:", float(baseline_df["RMSE_improve_%"].mean()))

print("\n✅ 리포트 끝! (feature_cols / 결측 / 요약통계 / DELTA&NEXT 성능 / baseline 비교)")
print("="*80)




Loaded: (9731, 39)
[Filter] YEAR>=2015 & PA>=223: 9731 -> 1146 (removed 8585)
Years in dataset: 2015 ~ 2024
Final X/Y: (823, 22) (823, 10)
Delta targets: ['AVG_delta', 'RBI_delta', 'HR_delta', 'SB_delta', 'OBP_delta', 'SLG_delta', 'OPS_delta', 'WAR_delta', 'wRC_plus_delta', 'PA_delta']
[Split] train < 2023, test >= 2023
Train samples: 668 Test samples: 155


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 22)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 256)            │         5,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,226 (192.29 KB)

 Trainable params: 48,458 (189.29 KB)

...

,target,BASE_MAE,MODEL_MAE,MAE_improve_%,BASE_RMSE,MODEL_RMSE,RMSE_improve_%
0,AVG_next,0.026684,0.024813,7.012344,0.033368,0.030464,8.704371
6,OPS_next,0.071342,0.066012,7.470712,0.091828,0.084405,8.084170
9,PA_next,92.296776,89.120407,3.441473,122.054931,112.369108,7.935626
8,wRC_plus_next,17.797421,16.827574,5.449367,23.219845,21.426238,7.724456
1,RBI_next,18.658064,17.616325,5.583315,24.219694,22.631385,6.557925
4,OBP_next,0.025206,0.024307,3.568181,0.032501,0.030454,6.297390
2,HR_next,5.406452,5.546183,-2.584522,7.427933,7.267376,2.161535
3,SB_next,5.367742,5.248017,2.230458,8.257157,8.114725,1.724957
7,WAR_next,1.321742,1.241628,6.061217,1.716650,1.689364,1.589479
5,SLG_next,0.052368,0.054933,-4.898894,0.067452,0.068523,-1.589000



Summary (mean improvement):
MAE improvement %: 3.333365089912436
RMSE improvement %: 4.919090818197706

✅ 리포트 끝! (feature_cols / 결측 / 요약통계 / DELTA&NEXT 성능 / baseline 비교)

[이유찬 12894] 2024 ACTUAL  →  2025 PREDICTED  →  2025 ACTUAL


,Type,Year,AVG,RBI,HR,SB,OBP,SLG,OPS,WAR,wRC_plus,PA
0,ACTUAL,2024,0.277,23.0,3.0,16.0,0.341,0.364,0.705,1.04,87.9,262.0


,Type,Year,AVG,RBI,HR,SB,OBP,SLG,OPS,WAR,wRC_plus,PA
0,PREDICTED,2025,0.268332,25.048622,4.403632,13.916182,0.338771,0.343291,0.692176,0.745311,85.69809,289.828339


,Type,Year,AVG,RBI,HR,SB,OBP,SLG,OPS,WAR,wRC_plus,PA
0,ACTUAL,2025,0.242,16.0,1.0,12.0,0.328,0.29,0.618,0.84,79.0,311.0
